In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [ ]:
url = "https://catalogo.movistar.com.pe/"
datos = []
dict = {}

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
equipos = driver.find_element(By.ID, 'tab-portabilidad').find_element(By.CLASS_NAME, 'list-portabilidad').find_elements(By.TAG_NAME, 'li')
print("Equipos encontrados ->", len(equipos))
#equipos = list(set(equipos)) # elimina miembros duplicados
links = [link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in equipos]

#for link in links:
 #   print(link)
driver.close()

In [ ]:
# Crear una lista que representará la tabla
tabla = []

# Agregar el encabezado de la tabla como el primer elemento de la lista
encabezado = ['marca', 'modelo', 'memoria', 'enlace', 'modalidad', 'planes', 'precio']
tabla.append(encabezado)

In [ ]:
def category():
    categorias = driver.find_element(By.CLASS_NAME, 'js_select_category')
    categorias.click()
    categorias = driver.find_element(By.CLASS_NAME, 'js_select_category').find_elements(By.TAG_NAME, 'option')
    for categoria in categorias:
        modalidad = categoria.text
        print(modalidad)
        categoria.click()
        
        if(categoria.text == 'Renovación'):
            close_button = driver.find_element(By.CLASS_NAME, 'modal-renovacion-validar__close')
            close_button.click()
            
        al_contado()
        
        plan_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js_select_plan')))
        plan_button.click()
        plan_button = driver.find_element(By.CLASS_NAME, 'js_select_plan').find_elements(By.TAG_NAME, 'option')
        
        # Hay que crear una lista y guardar en esa lista cada diccionario dict
        for plan in plan_button:
            nombre_plan = plan.text
            print(nombre_plan)
            plan = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js_select_plan')))
            plan.click() #Elige plan
            #print(plan.text)
            price = driver.find_element(By.CLASS_NAME, 'js_price_cash')       
            precio = price.text
            print(precio)
            fila = [marca, modelo, memoria, enlace, modalidad, nombre_plan, precio]
            tabla.append(fila)
        
        
        

In [ ]:
def scroll():
    # Crear una instancia de ActionChains
    action = ActionChains(driver)
    view_button = driver.find_element(By.CLASS_NAME, 'view-btn-desktop')
    # Realizar acción de desplazamiento usando move_to_element
    action.move_to_element(view_button ).perform()

In [ ]:
def al_contado():
    fee_button = driver.find_element(By.CLASS_NAME, 'js_select_fee')
    fee_button.click()
    fee_button = driver.find_element(By.CLASS_NAME, 'js_select_fee').find_elements(By.TAG_NAME, 'option')
    fee_button[0].click() #Elige Al Contado

In [ ]:
def agregar_fila():
    fila = [marca, modelo, memoria, enlace, modalidad, nombre_plan, precio]
    tabla.append(fila)

In [ ]:
def planes():
    plan_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js_select_plan')))
    plan_button.click()
    plan_button = driver.find_element(By.CLASS_NAME, 'js_select_plan').find_elements(By.TAG_NAME, 'option')
    
    # Hay que crear una lista y guardar en esa lista cada diccionario dict
    for plan in plan_button:
        nombre_plan = plan.text
        print(nombre_plan)
        plan = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js_select_plan')))
        plan.click() #Elige plan
        #print(plan.text)
        price = driver.find_element(By.CLASS_NAME, 'js_price_cash')       
        precio = price.text
        print(precio)
        fila = [marca, modelo, memoria, enlace, modalidad, nombre_plan, precio]
        tabla.append(fila)
        
        

In [ ]:
driver = webdriver.Chrome()
driver.get(links[1]) #28
enlace = links[1] # revisar 10 ElementNotInteractableException , 0
print(enlace)

#x_button = driver.find_element(By.CLASS_NAME, 'modals-header').find_elements(By.TAG_NAME, 'button')
marca = driver.find_element(By.CLASS_NAME, 'title-device').get_attribute("data-marca")
print(marca)

modelo = driver.find_element(By.CLASS_NAME, 'inner').find_element(By.TAG_NAME, 'h1').text
print(modelo)

#memories = driver.find_element(By.CLASS_NAME, 'memory').find_elements(By.TAG_NAME, 'span')

#memories = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'memory')))
#memories.click()
memories = driver.find_element(By.CLASS_NAME, 'memory').find_elements(By.TAG_NAME, 'span')
#print(len(memories))
if(len(memories) > 1 ):
    memories[-1].click() # ayuda a limpiar la sobreposicion de elementos
    for memoria in memories: 
        print(memoria.text)
        try:
            memoria.click()
            memoria = memoria.text
            scroll()
            category()
        except Exception as err:
            #print(err)
            print("El equipo no está disponible.")

else:
    memoria = memories[0].text
    print(memoria)
    scroll()
    category()


driver.quit()

print(tabla)


In [ ]:
df = pd.DataFrame(tabla)

nombre_archivo_excel = 'movistar-planes.xlsx'
df.to_excel(nombre_archivo_excel)

print(f"DataFrame guardado en el archivo '{nombre_archivo_excel}'.")